# Como a programação dinâmica é usada

A programação dinâmica, no contexto de alinhamento de sequências, é uma técnica que resolve problemas complexos quebrando-os em subproblemas menores e armazenando as soluções desses subproblemas para evitar recálculos.

No caso do alinhamento, o objetivo é encontrar a melhor correspondência entre duas sequências, considerando operações como inserções, deleções e substituições de caracteres. A programação dinâmica aborda esse problema construindo uma matriz onde cada célula representa a pontuação do melhor alinhamento possível até aquele ponto nas duas sequências.

Cada célula da matriz é preenchida com base na pontuação das células adjacentes, considerando o custo de cada operação (match, mismatch, gap). A pontuação final, representando o melhor alinhamento, é encontrada na última célula da matriz. O caminho de volta através da matriz, a partir da última célula, revela o alinhamento ótimo em si.

Essencialmente, a programação dinâmica permite encontrar a solução ótima explorando todas as combinações possíveis de alinhamento de forma eficiente, armazenando e reutilizando resultados parciais.

# Código

In [5]:
def smith_waterman(seq1, seq2, match_score=1, mismatch_penalty=-1, gap_penalty=-1):
    m, n = len(seq1), len(seq2)
    matrix = [[0] * (n + 1) for _ in range(m + 1)]

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            match = matrix[i - 1][j - 1] + (match_score if seq1[i - 1] == seq2[j - 1] else mismatch_penalty)
            delete = matrix[i - 1][j] + gap_penalty
            insert = matrix[i][j - 1] + gap_penalty
            matrix[i][j] = max(0, match, delete, insert)

    # Encontrar a pontuação máxima e sua posição na matriz
    max_score = 0
    max_i, max_j = 0, 0
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if matrix[i][j] > max_score:
                max_score = matrix[i][j]
                max_i, max_j = i, j

    # Rastrear o alinhamento ótimo
    alignment1, alignment2 = "", ""
    i, j = max_i, max_j
    while matrix[i][j] != 0:
        if matrix[i][j] == matrix[i - 1][j - 1] + (match_score if seq1[i - 1] == seq2[j - 1] else mismatch_penalty):
            alignment1 = seq1[i - 1] + alignment1
            alignment2 = seq2[j - 1] + alignment2
            i -= 1
            j -= 1
        elif matrix[i][j] == matrix[i - 1][j] + gap_penalty:
            alignment1 = seq1[i - 1] + alignment1
            alignment2 = "-" + alignment2
            i -= 1
        else:
            alignment1 = "-" + alignment1
            alignment2 = seq2[j - 1] + alignment2
            j -= 1

    return max_score, matrix, alignment1, alignment2

# Matriz de Score

In [11]:
smith_waterman("-CGTGAATTCAT", "-GACTTAC", match_score = 5, mismatch_penalty = -3, gap_penalty = -4)[1]

[[0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 5, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 2, 0, 5, 1, 0, 0, 5],
 [0, 0, 6, 2, 1, 2, 0, 0, 1],
 [0, 0, 2, 3, 0, 6, 7, 3, 0],
 [0, 0, 5, 1, 0, 2, 3, 4, 0],
 [0, 0, 1, 10, 6, 2, 0, 8, 4],
 [0, 0, 0, 6, 7, 3, 0, 5, 5],
 [0, 0, 0, 2, 3, 12, 8, 4, 2],
 [0, 0, 0, 0, 0, 8, 17, 13, 9],
 [0, 0, 0, 0, 5, 4, 13, 14, 18],
 [0, 0, 0, 5, 1, 2, 9, 18, 14],
 [0, 0, 0, 1, 2, 6, 7, 14, 15]]

# Alinhamento e Score

In [21]:
alignment1, alignment2 = smith_waterman("-CGTGAATTCAT", "-GACTTAC", match_score = 5, mismatch_penalty = -3, gap_penalty = -4)[2:]

In [25]:
for (i, j) in zip(alignment1, alignment2):
    if i == j:
      print("+5")
    elif i == "-" or j == "-":
      print("-4")
    else:
      print("-3")

+5
+5
-3
+5
+5
-4
+5


# Teste de alinhamento

In [26]:
smith_waterman("GAATTCAGTTA", "GGATCGA", match_score = 5, mismatch_penalty = -3, gap_penalty = -4)

(14,
 [[0, 0, 0, 0, 0, 0, 0, 0],
  [0, 5, 5, 1, 0, 0, 5, 1],
  [0, 1, 2, 10, 6, 2, 1, 10],
  [0, 0, 0, 7, 7, 3, 0, 6],
  [0, 0, 0, 3, 12, 8, 4, 2],
  [0, 0, 0, 0, 8, 9, 5, 1],
  [0, 0, 0, 0, 4, 13, 9, 5],
  [0, 0, 0, 5, 1, 9, 10, 14],
  [0, 5, 5, 1, 2, 5, 14, 10],
  [0, 1, 2, 2, 6, 2, 10, 11],
  [0, 0, 0, 0, 7, 3, 6, 7],
  [0, 0, 0, 5, 3, 4, 2, 11]],
 'GAATTC-A',
 'GGA-TCGA')